In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import faiss

model = SentenceTransformer("cointegrated/rubert-tiny2")

In [4]:
df = pd.read_csv("/home/artemiy/recsys_project/data/dataset_clean_new.csv")
df.head()

,url,poster,title,description,rating,year,genres,cast,country,age
0,https://kino.mail.ru/series_915957_potapov_i_l...,https://resizer.mail.ru/p/a4bff38f-c8db-57ac-9...,Гадалка,"Люся видит во сне мертвое тело своей сестры, б...",6.4,2019,"Драма, Детектив, Мистика","Михаил Пореченков, Екатерина Олькина, Яна Край...",Россия,16
1,https://kino.mail.ru/series_773009_ohotnik/,https://resizer.mail.ru/p/dccc9300-db25-55ad-b...,Охотник,"Средневековый охотник — Владимир, сын Казатула...",7.4,2006,"Фантастика, Фэнтези, Приключения","Евгений Цыганов, Елена Панова, Сергей Никоненк...",Россия,16
2,https://kino.mail.ru/series_925738_velikaya/,https://resizer.mail.ru/p/2bebfd49-f476-5000-9...,Великая,Сериал о молодости императрицы Екатерины II. М...,4.8,2020,"Драма, Комедия","Эль Фаннинг, Николас Холт, Фиби Фокс, Адам Год...",США,18
3,https://kino.mail.ru/series_784898_fantastiche...,https://resizer.mail.ru/p/68617b86-fd8b-5e43-b...,Фантастическая девушка,Главная героиня сериала — Эви — фантастическая...,6.5,1987,"Фантастика, Комедия, Семейный","Донна Пескоу, Морин Флэннигэн, Берт Рейнолдс, ...",США,0
4,https://kino.mail.ru/series_785356_shpionka/,https://resizer.mail.ru/p/733fedb7-047d-58d5-9...,Шпионка,"Переезды по всему миру, борьба с «плохими парн...",7.1,2001,"Драма, Боевик, Криминал, Триллер, Детектив","Дженнифер Гарнер, Майкл Вартан, Рон Рифкин, Ви...",США,16


In [5]:
# Функция для предобработки текстовых столбцов
def preprocess_text_column(column):
    df[column] = df[column].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
    df[column] = df[column].fillna('')  # Замена NaN на пустую строку
    df[column] = df[column].apply(str)  # Преобразование всех значений в строки

# Применение функции к каждому текстовому столбцу
text_columns = ['title', 'description', 'genres', 'cast']
for col in text_columns:
    preprocess_text_column(col)

# Векторизация текстовых данных и объединение в один массив эмбеддингов
all_embeddings = np.hstack([model.encode(df[col].values.tolist()) for col in text_columns])

In [6]:
# Векторизация текстовых данных и объединение в один массив эмбеддингов
# Используем np.mean для получения среднего эмбеддинга по всем столбцам
all_embeddings = np.array([model.encode(df[col].values.tolist()) for col in text_columns])
all_embeddings = np.mean(all_embeddings, axis=0)

# Создание индекса FAISS
index = faiss.IndexFlatIP(all_embeddings.shape[1])

# Добавление эмбеддингов в индекс
index.add(all_embeddings)

ValueError: input not a numpy array

In [ ]:
# Кодирование пользовательского запроса
user_input = 'Фильм про спортсменов'
user_input_embedding = model.encode(user_input).reshape(1, -1)

# Поиск ближайших соседей
D, I = index.search(user_input_embedding, 5)

# Вывод результатов
print('Результаты поиска (индексы):', I)
print('Результаты поиска (расстояния):', D)

# Использование косинусного сходства для дополнительной проверки
cos_sim_scores = util.pytorch_cos_sim(user_input_embedding, all_embeddings)

# Вывод косинусных сходств
print('Косинусное сходство:', cos_sim_scores)

Результаты поиска (индексы): [[ 7367 11612 11451  1011  8530]]
Результаты поиска (расстояния): [[0.62535715 0.62425184 0.61865187 0.6181731  0.6181023 ]]
Косинусное сходство: tensor([[0.6387, 0.6372, 0.6798,  ..., 0.6329, 0.6461, 0.5909]])
